In [1]:
import vectorbtpro as vbt
import numpy as np
import pandas as pd
from numba import njit

In [113]:
import warnings

warnings.filterwarnings("ignore", category=UserWarning)

## Data

Using GBM generated data to ensure consistency

In [3]:
data = vbt.GBMOHLCData.pull(
    symbols=['CHADCOIN'],
    start='2020-01-01 UTC',
    end='2021-01-01 UTC',
    seed=vbt.symbol_dict(BTCUSD=42),
    n_ticks=1000,
)

# Creating Splitters

- Sets / Splits / Ranges

In [121]:
splitter = vbt.Splitter.from_n_rolling(
    index=data.index,
    n=5,
    split=0.5,
    set_labels=["train","test"],
)
splitter.plot()

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgba(31,119,180,0.8000)'], [1.0,
                             'rgba(31,119,180,0.8000)']],
              'hoverongaps': False,
              'hovertemplate': '%{x}<br>Split: %{y}<br>Set: train',
              'legendgroup': 'train',
              'name': 'train',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': '562dab7b-117c-4863-9ceb-145071326dea',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 3, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2020, 12, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 12, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020,

In [30]:
splitter2 = vbt.Splitter.from_n_rolling(
    index=data.index,
    n=5,
    #length="180 days",
    split=(0.5,0.2,0.3),
    set_labels=["train","validation","test"],
)
splitter2.plot()

/home/chad/Desktop/videos/vbt-cross-validation/venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/home/chad/Desktop/videos/vbt-cross-validation/venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant

/home/chad/Desktop/videos/vbt-cross-validation/venv/lib/python3.12/site-packages/jupyter_client/session.py:721: UserWarning:

Message serialization failed with:
Out of range float values are not JSON compliant: nan
Supporting this message is deprecated in jupyter-client 7, please make sure your message is JSON-compliant



FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgba(31,119,180,0.8000)'], [1.0,
                             'rgba(31,119,180,0.8000)']],
              'hoverongaps': False,
              'hovertemplate': '%{x}<br>Split: %{y}<br>Set: train',
              'legendgroup': 'train',
              'name': 'train',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': 'b5b4973c-4ef5-4e48-8aa7-1a098c5d197b',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 2, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 3, 0, 0, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2020, 12, 29, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 12, 30, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020,

# Applying Splitters

In [6]:
splitter2.take(data)[4, "test"].close

symbol,CHADCOIN
2020-09-08 00:00:00+00:00,144.623461
2020-09-09 00:00:00+00:00,134.573582
2020-09-10 00:00:00+00:00,127.158331
2020-09-11 00:00:00+00:00,124.563202
2020-09-12 00:00:00+00:00,122.899317
2020-09-13 00:00:00+00:00,122.432707
2020-09-14 00:00:00+00:00,123.751676
2020-09-15 00:00:00+00:00,127.119288
2020-09-16 00:00:00+00:00,123.857773


## Simple UDFs

In [7]:
def buy_and_hold_return(data):
    return vbt.Portfolio.from_holding(data).total_return

In [8]:
results = splitter2.apply(
    buy_and_hold_return,
    vbt.Takeable(data),
    merge_func="concat"
)
results

split  set         symbol  
0      train       CHADCOIN   -0.141987
       validation  CHADCOIN   -0.054555
       test        CHADCOIN    0.054051
1      train       CHADCOIN    0.077356
       validation  CHADCOIN    0.079714
       test        CHADCOIN    0.017928
2      train       CHADCOIN    0.079998
       validation  CHADCOIN   -0.056170
       test        CHADCOIN    0.089008
3      train       CHADCOIN   -0.032343
       validation  CHADCOIN    0.064377
       test        CHADCOIN   -0.018575
4      train       CHADCOIN    0.218608
       validation  CHADCOIN   -0.009936
       test        CHADCOIN   -0.143585
5      train       CHADCOIN   -0.021707
       validation  CHADCOIN    0.021110
       test        CHADCOIN   -0.106084
Name: total_return, dtype: float64

## Dynamic Splitter creation

In [9]:
@vbt.split(
    splitter="from_n_rolling",
    splitter_kwargs=dict(n=5, split=0.5, set_labels=["train","test"]),
    takeable_args=["data"],
    merge_func="reset_column_stack",
    attach_bounds="index",
)
def buy_and_hold_return_series(data):
    return data.returns

In [10]:
buy_and_hold_return_series(data)

split                          0                            \
set                        train                      test   
start  2020-01-01 00:00:00+00:00 2020-02-06 00:00:00+00:00   
end    2020-02-06 00:00:00+00:00 2020-03-14 00:00:00+00:00   
symbol                  CHADCOIN                  CHADCOIN   
0                       0.000000                  0.000000   
1                      -0.004735                  0.010826   
2                       0.005992                 -0.003721   
3                      -0.034626                 -0.040257   
4                      -0.026014                  0.011202   
5                      -0.040578                 -0.004552   
6                      -0.023394                 -0.028222   
7                      -0.047874                 -0.023518   
8                      -0.053722                  0.047355   
9                       0.022638                 -0.004342   
10                     -0.016460                 -0.008957   
11                     -0.024432                  0.002893   
12                     -0.007204                 -0.004856   
13                     -0.005297                  0.000724   
14                     -0.043660                  0.025167   
15                     -0.045970                 -0.003871   
16                     -0.004924                  0.018165   
17                      0.023128                 -0.015218   
18                     -0.042359                  0.039143   
19                      0.017847                 -0.017056   
20                      0.017524                 -0.017623   
21                      0.045270                 -0.016953   
22                      0.014438                  0.025711   
23                      0.015200                  0.005443   
24                      0.036393                 -0.027123   
25                      0.002335                 -0.018966   
26                     -0.010726                  0.040607   
27                     -0.014874                 -0.011870   
28                      0.011095                 -0.030630   
29                     -0.035967                 -0.016859   
30                      0.037652                  0.013669   
31                     -0.039155                  0.029730   
32                      0.012457                 -0.004781   
33                      0.017468                  0.016672   
34                      0.062612                  0.029153   
35                      0.043136                  0.026621   
36                           NaN                  0.073336   

split                          1                            \
set                        train                      test   
start  2020-03-14 00:00:00+00:00 2020-04-19 00:00:00+00:00   
end    2020-04-19 00:00:00+00:00 2020-05-26 00:00:00+00:00   
symbol                  CHADCOIN                  CHADCOIN   
0                       0.000000                  0.000000   
1                      -0.018737                 -0.015834   
2                       0.015123                  0.021469   
3                       0.030173                  0.040755   
4                      -0.033697                  0.017959   
5                       0.011672                 -0.033955   
6                      -0.021777                  0.051364   
7                       0.044854                 -0.042426   
8                      -0.051222                 -0.032862   
9                      -0.026101                 -0.030142   
10                      0.013374                  0.036562   
11                      0.028768                  0.025468   
12                     -0.019328                  0.000551   
13                      0.018775                 -0.005214   
14                     -0.012537                 -0.013913   
15                      0.002540                  0.040869   
16                      0.003253                 -0.026729   
17                      0.05212

In [11]:
buy_and_hold_return_series(data, _apply_kwargs=dict(set_="train", merge_func="column_stack", attach_bounds=None) )

split,0,1,2,3,4
symbol,CHADCOIN,CHADCOIN,CHADCOIN,CHADCOIN,CHADCOIN
2020-01-01 00:00:00+00:00,0.000000,NaN,NaN,NaN,NaN
2020-01-02 00:00:00+00:00,-0.004735,NaN,NaN,NaN,NaN
2020-01-03 00:00:00+00:00,0.005992,NaN,NaN,NaN,NaN
2020-01-04 00:00:00+00:00,-0.034626,NaN,NaN,NaN,NaN
2020-01-05 00:00:00+00:00,-0.026014,NaN,NaN,NaN,NaN
...,...,...,...,...,...
2020-11-19 00:00:00+00:00,NaN,NaN,NaN,NaN,0.041713
2020-11-20 00:00:00+00:00,NaN,NaN,NaN,NaN,-0.006273
2020-11-21 00:00:00+00:00,NaN,NaN,NaN,NaN,0.007597


# Pipelines

In [302]:
data2 = vbt.GBMOHLCData.pull(
    symbols=['CHADCOIN'],
    start='2020-01-01 UTC',
    end='2021-01-01 UTC', # '2030-01-01 UTC'
    seed=vbt.symbol_dict(CHADCOIN=10),
    timeframe="15 min",
    n_ticks=20,
)
#data2.close.plot()

In [104]:
@vbt.cv_split(
    merge_func="concat",
    splitter="from_n_rolling",
    splitter_kwargs=dict(n=4, split=0.5, set_labels=["train","test"]),
    takeable_args=["data"],
    return_grid='all',
)
def rsi_pipeline(data, upper_threshold=70, lower_threshold=30):
    rsi = vbt.RSI.run(close=data).rsi
    entries = rsi.vbt.crossed_below(lower_threshold)
    exits = rsi.vbt.crossed_above(upper_threshold)
    pf = vbt.Portfolio.from_signals(
        data,
        long_entries=entries,
        long_exits=exits,
    )

    if len(pf.trades) < 1:
        return vbt.NoResult

    return pf.sharpe_ratio

In [105]:
grid, best = rsi_pipeline(
    data2.close["CHADCOIN"],
    vbt.Param(range(70, 80, 1), name="Upper"),
    vbt.Param(range(20, 30, 1), name="Lower"),
)

 25%|##5       | 1/4 [00:04<00:13,  4.39s/it, split=1]

In [106]:
best.xs("test", axis=0, level="set")

split  Upper  Lower
0      71     23       1.856017
1      76     20      -0.012611
2      79     29       0.985265
3      72     25       3.224272
dtype: float64

## Compare to buy and hold

In [107]:
splitter3 = rsi_pipeline(
    data2,
    _return_splitter=True
)
#splitter3.plot()

In [108]:
stacked_data = splitter3.take(data2.close, into="reset_stacked")
benchmark_sharpe = vbt.Portfolio.from_holding(stacked_data, freq="15m").sharpe_ratio
benchmark_sharpe

split  set    symbol  
0      train  CHADCOIN   -2.459066
       test   CHADCOIN   -1.588828
1      train  CHADCOIN   -0.049191
       test   CHADCOIN    2.743796
2      train  CHADCOIN    3.918398
       test   CHADCOIN   -0.308933
3      train  CHADCOIN    1.149608
       test   CHADCOIN   -1.683509
Name: sharpe_ratio, dtype: float64

In [109]:
benchmark_sharpe.xs("test", axis=0, level="set")

split  symbol  
0      CHADCOIN   -1.588828
1      CHADCOIN    2.743796
2      CHADCOIN   -0.308933
3      CHADCOIN   -1.683509
Name: sharpe_ratio, dtype: float64

In [110]:
best.xs("test", axis=0, level="set")

split  Upper  Lower
0      71     23       1.856017
1      76     20      -0.012611
2      79     29       0.985265
3      72     25       3.224272
dtype: float64

## Indicator Warmup time

In [357]:
def custom_split_func(index, prev_start, prev_end, test_weeks=2, initial_offset=14):
    if prev_start is None:
        prev_start = index[0] + index.freq * initial_offset
        if not (prev_start.day == 1 and prev_start.time() == pd.Timestamp("00:00:00").time()):
            prev_start = (prev_start + pd.DateOffset(months=1)).normalize()

        prev_end = prev_start + pd.offsets.MonthBegin(1)

    new_end = prev_end + pd.offsets.MonthBegin(1)
    if new_end > index[-1]:
        return None
    return [
        slice(prev_start, new_end - pd.Timedelta(weeks=test_weeks)),  
        slice(new_end - pd.Timedelta(weeks=test_weeks), new_end)
    ]

vbt.Splitter.from_split_func(
    data2.index,
    split_func=custom_split_func,
    split_args=(vbt.Rep("index"), vbt.Rep("prev_start"), vbt.Rep("prev_end")),
    split_kwargs=dict(test_weeks=4, initial_offset=14),
    range_bounds_kwargs=dict(index_bounds=True),
    set_labels=["test","train"]
).plot()

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgba(31,119,180,0.8000)'], [1.0,
                             'rgba(31,119,180,0.8000)']],
              'hoverongaps': False,
              'hovertemplate': '%{x}<br>Split: %{y}<br>Set: test',
              'legendgroup': 'test',
              'name': 'test',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': 'af0f5c25-3644-4dc9-abcb-1b3dfab74e7f',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 1, 0, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 1, 0, 30, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2020, 12, 31, 23, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 12, 31, 23, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(20

In [ ]:
@vbt.cv_split(
    merge_func="concat",
    splitter="from_split_func",
    splitter_kwargs=dict(
        split_func=custom_split_func,
        split_args=(vbt.Rep("index"), vbt.Rep("prev_start"), vbt.Rep("prev_end")),
        split_kwargs=dict(test_weeks=4, initial_offset=14),
        range_bounds_kwargs=dict(index_bounds=True),
        set_labels=["train","test"]
    ),
    takeable_args=["data"],
    return_grid='all',
)
def rsi_pipeline(data, upper_threshold=70, lower_threshold=30):
    window_length=14
    rsi = vbt.RSI.run(close=data, window=window_length).rsi
    entries = rsi.vbt.crossed_below(lower_threshold)
    exits = rsi.vbt.crossed_above(upper_threshold)
    pf = vbt.Portfolio.from_signals(
        data,
        long_entries=entries,
        long_exits=exits,
    )



    if len(pf.trades) < 1:
        return vbt.NoResult
    
    print(pf.sharpe_ratio)

    return pf.sharpe_ratio

In [250]:
splitter = rsi_pipeline(
    data2.close["CHADCOIN"],
    vbt.Param(range(70, 80, 1), name="Upper"),
    vbt.Param(range(20, 30, 1), name="Lower"),
    _splitter_kwargs=dict(split_kwargs=dict(test_weeks=2, initial_offset=14)),
    _return_splitter=True
)
splitter.plot()

FigureWidget({
    'data': [{'colorscale': [[0.0, 'rgba(31,119,180,0.8000)'], [1.0,
                             'rgba(31,119,180,0.8000)']],
              'hoverongaps': False,
              'hovertemplate': '%{x}<br>Split: %{y}<br>Set: test',
              'legendgroup': 'test',
              'name': 'test',
              'showlegend': True,
              'showscale': False,
              'type': 'heatmap',
              'uid': '07281e32-a602-4106-ad1c-1e33d5193e82',
              'x': array([datetime.datetime(2020, 1, 1, 0, 0, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 1, 0, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 1, 1, 0, 30, tzinfo=datetime.timezone.utc), ...,
                          datetime.datetime(2020, 12, 31, 23, 15, tzinfo=datetime.timezone.utc),
                          datetime.datetime(2020, 12, 31, 23, 30, tzinfo=datetime.timezone.utc),
                          datetime.datetime(20

In [386]:
grid, best = rsi_pipeline(
    data2.close["CHADCOIN"],
    71,
    21,
    _splitter_kwargs=dict(split_kwargs=dict(test_weeks=1, initial_offset=14)),
    _apply_kwargs=dict(split=1)
)
best

2.347230231136428
2.347230231136428
2.347230231136428
16.429512169799573
16.429512169799573


set
train     2.347230
test     16.429512
dtype: float64

## Indicator Warmup

In [396]:
@vbt.cv_split(
    merge_func="concat",
    splitter="from_split_func",
    splitter_kwargs=dict(
        split_func=custom_split_func,
        split_args=(vbt.Rep("index"), vbt.Rep("prev_start"), vbt.Rep("prev_end")),
        split_kwargs=dict(test_weeks=4, initial_offset=14),
        range_bounds_kwargs=dict(index_bounds=True),
        set_labels=["train","test"]
    ),
    takeable_args=None, #!
    index_from="data", #!
    return_grid=True,
)
def rsi_pipeline_warmup(range_, data, upper_threshold=70, lower_threshold=30):

    window_length=14
    buffered_range = slice(range_.start - window_length, range_.stop)  
    data_buffered = data.iloc[buffered_range]
    
    rsi = vbt.RSI.run(close=data_buffered, window=window_length).rsi
    entries_buffered = rsi.vbt.crossed_below(lower_threshold)
    exits_buffered = rsi.vbt.crossed_above(upper_threshold)

    pf = vbt.Portfolio.from_signals(
        data.iloc[window_length:],
        long_entries=entries_buffered.iloc[window_length:],
        long_exits=exits_buffered.iloc[window_length:],
    )

    if len(pf.trades) < 1:
        return vbt.NoResult
    
    print(pf.sharpe_ratio)

    return pf.sharpe_ratio

In [397]:
grid, best = rsi_pipeline_warmup(
    vbt.Rep("range_"),
    data2.close["CHADCOIN"],
    71,
    21,
    _splitter_kwargs=dict(split_kwargs=dict(test_weeks=1, initial_offset=14)),
    _apply_kwargs=dict(split=1)
)
best

1.1179835310478663
1.1179835310478663
2.26568663783191


set
train    1.117984
test     2.265687
dtype: float64